In [36]:
!pip install wandb
!pip install scikit-learn
!pip install pandas
!pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 MB 4.0 MB/s eta 0:00:0000:0100:01
  Using cached sympy-1.11.1-py3-none-any.whl (6.5 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 3.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 3.2 MB/s eta 0:00:00a 0:00:01


In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_breast_cancer, fetch_20newsgroups
from sklearn.model_selection import KFold, train_test_split
import wandb
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, log_loss
import numpy as np
from simpletransformers.classification import MultiLabelClassificationArgs, MultiLabelClassificationModel

# Loading in data

In [40]:
data = load_breast_cancer()

In [22]:
print(data.feature_names)

['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']


In [23]:
print(data.target_names)

['malignant' 'benign']


In [41]:
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Basic Example with Pretty Graphs

In [6]:
def main():
#     Call wandb.init() to start server
    wandb.init(project="pretty_graphs_run")
#     Instantiate model
    model = RandomForestClassifier(random_state=42)
#     Fit model with training data
    model.fit(X_train, y_train)
#     Make predictions
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)
#     Generate output plots
    wandb.sklearn.plot_confusion_matrix(y_test, y_pred, data.target_names)
    wandb.sklearn.plot_learning_curve(model, X, y)
    wandb.sklearn.plot_classifier(model, X_train, X_test, y_train, y_test, y_pred, y_prob, data.target_names,
                                                         model_name='RF', feature_names=data.feature_names)
    wandb.sklearn.plot_precision_recall(y_test, y_prob, data.target_names)
    wandb.sklearn.plot_feature_importances(model, data.feature_names)
#     Compute performance
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
#     Log results
    wandb.log(
        {
            "F1-Score": f1,
            "Precision": precision,
            "Recall": recall
        }
    )
#     Close server
    wandb.finish()

In [7]:
main()

wandb: Currently logged in as: scott-clare. Use `wandb login --relogin` to force relogin


wandb: 
wandb: Plotting RF.
/Users/scottclare/miniconda3/envs/tech_discussion_group/lib/python3.8/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
wandb: Logged feature importances.
wandb: Logged confusion matrix.
/Users/scottclare/miniconda3/envs/tech_discussion_group/lib/python3.8/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
wandb: Logged summary metrics.
wandb: Logged class proportions.
/Users/scottclare/miniconda3/envs/tech_discussion_group/lib/python3.8/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/Users/scottclare/miniconda3/envs/tech_discussion_group/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs faile

F1-Score,▁
Precision,▁
Recall,▁
F1-Score,0.95105
Precision,0.94444
Recall,0.95775


# Plotting metrics every step/iteration

In [43]:
def main():
#     Call wandb.init() to start server
    with wandb.init(project="logistic_regression") as run:

        n_iters = 300
        learning_rate = 0.001
    
        def _sigmoid(x):
            return 1/(1+np.exp(-x))

        n_samples, n_features = X_train.shape
        weights = np.zeros(n_features)
        bias = 0

        for iteration in range(n_iters):
            linear_pred = np.dot(X_train, weights) + bias
            probabilities = _sigmoid(linear_pred)
            predictions = [0 if prob <= 0.5 else 1 for prob in probabilities]

            dw = (1 / n_samples) * np.dot(X_train.T, (probabilities - y_train))
            db = (1 / n_samples) * np.sum(probabilities - y_train)
            
            weights = weights - learning_rate * dw
            bias = bias - learning_rate * db
            wandb.log(
                {
                    "log_loss": log_loss(y_train, probabilities),
                    "accuracy": accuracy_score(y_train, predictions)
                }
            )

        linear_pred = np.dot(X_test, weights) + bias
        y_prob = _sigmoid(linear_pred)
        y_pred = [0 if prob<=0.5 else 1 for prob in y_prob]

    #     Generate output plots
        wandb.sklearn.plot_confusion_matrix(y_test, y_pred, data.target_names)

    #     Compute performance
        f1 = f1_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
#         Log results
        wandb.log(
            {
                "F1-Score": f1,
                "Precision": precision,
                "Recall": recall
            }
        )

In [44]:
main()

/var/folders/1l/ry3dd_s55j13rdvlhgfn7kv00000gn/T/ipykernel_89305/352022253.py:9: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
/var/folders/1l/ry3dd_s55j13rdvlhgfn7kv00000gn/T/ipykernel_89305/352022253.py:9: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
/var/folders/1l/ry3dd_s55j13rdvlhgfn7kv00000gn/T/ipykernel_89305/352022253.py:9: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
/var/folders/1l/ry3dd_s55j13rdvlhgfn7kv00000gn/T/ipykernel_89305/352022253.py:9: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
/var/folders/1l/ry3dd_s55j13rdvlhgfn7kv00000gn/T/ipykernel_89305/352022253.py:9: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
/var/folders/1l/ry3dd_s55j13rdvlhgfn7kv00000gn/T/ipykernel_89305/352022253.py:9: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
/var/folders/1l/ry3dd_s55j13rdvlhgfn7kv00000gn/T/ipykernel_89305/352022253.py:9: RuntimeWarnin

/var/folders/1l/ry3dd_s55j13rdvlhgfn7kv00000gn/T/ipykernel_89305/352022253.py:9: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
/var/folders/1l/ry3dd_s55j13rdvlhgfn7kv00000gn/T/ipykernel_89305/352022253.py:9: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
/var/folders/1l/ry3dd_s55j13rdvlhgfn7kv00000gn/T/ipykernel_89305/352022253.py:9: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
/var/folders/1l/ry3dd_s55j13rdvlhgfn7kv00000gn/T/ipykernel_89305/352022253.py:9: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
/var/folders/1l/ry3dd_s55j13rdvlhgfn7kv00000gn/T/ipykernel_89305/352022253.py:9: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
/var/folders/1l/ry3dd_s55j13rdvlhgfn7kv00000gn/T/ipykernel_89305/352022253.py:9: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
/var/folders/1l/ry3dd_s55j13rdvlhgfn7kv00000gn/T/ipykernel_89305/352022253.py:9: RuntimeWarnin

/var/folders/1l/ry3dd_s55j13rdvlhgfn7kv00000gn/T/ipykernel_89305/352022253.py:9: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
/var/folders/1l/ry3dd_s55j13rdvlhgfn7kv00000gn/T/ipykernel_89305/352022253.py:9: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
/var/folders/1l/ry3dd_s55j13rdvlhgfn7kv00000gn/T/ipykernel_89305/352022253.py:9: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
/var/folders/1l/ry3dd_s55j13rdvlhgfn7kv00000gn/T/ipykernel_89305/352022253.py:9: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
/var/folders/1l/ry3dd_s55j13rdvlhgfn7kv00000gn/T/ipykernel_89305/352022253.py:9: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
/var/folders/1l/ry3dd_s55j13rdvlhgfn7kv00000gn/T/ipykernel_89305/352022253.py:9: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
/var/folders/1l/ry3dd_s55j13rdvlhgfn7kv00000gn/T/ipykernel_89305/352022253.py:9: RuntimeWarnin

/var/folders/1l/ry3dd_s55j13rdvlhgfn7kv00000gn/T/ipykernel_89305/352022253.py:9: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
/var/folders/1l/ry3dd_s55j13rdvlhgfn7kv00000gn/T/ipykernel_89305/352022253.py:9: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
/var/folders/1l/ry3dd_s55j13rdvlhgfn7kv00000gn/T/ipykernel_89305/352022253.py:9: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
/var/folders/1l/ry3dd_s55j13rdvlhgfn7kv00000gn/T/ipykernel_89305/352022253.py:9: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
/var/folders/1l/ry3dd_s55j13rdvlhgfn7kv00000gn/T/ipykernel_89305/352022253.py:9: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
/var/folders/1l/ry3dd_s55j13rdvlhgfn7kv00000gn/T/ipykernel_89305/352022253.py:9: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-x))
/var/folders/1l/ry3dd_s55j13rdvlhgfn7kv00000gn/T/ipykernel_89305/352022253.py:9: RuntimeWarnin

F1-Score,▁
Precision,▁
Recall,▁
accuracy,▁▁▄▅▆▅▅▅▅▅▆▅▅▅▅▆▆▆▆▇████████████████████
log_loss,██▁▅▃▄▄▄▄▄▄▄▄▄▄▄▃▃▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
F1-Score,0.91852
Precision,0.89855
Recall,0.93939
accuracy,0.91429
log_loss,2.4003
